# Data Pre-Processing and Creating HDF5 Store
In this file, we use the Annotation and Recording files to sample the Raw Image and corresponding Gaze Data using the frame number and the timestamp. Then using the obtained Raw Image we generate the opticalFlow Image.

Once the data is extracted, we then store them in a HDF5 file.

In [ ]:
#import the required modules
%pylab inline
import numpy as np
import cv2
import h5py
import tensorflow as tf
import pandas as pd
from keras.preprocessing import image
from IPython.display import clear_output
from pathlib import Path

In [ ]:
# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.1

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [3]:
#choose the path for the HDF5 file
HDF5_PATH = '/home/gururajaramesh/completedata.hdf5'

In [ ]:
#choose the person whose data has to be sampled
person = 4

In [4]:
#Get the total frame count and frames per second of the person's recording
vid = cv2.VideoCapture("/bigpool/UbiComp2015/Recordings/P0"+ person +"_2014_05_29/4Recording/world_Rec.mkv")
total_frame = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(vid.get(cv2.CAP_PROP_FPS))
print ('total frame in vid_5 is %d' %total_frame)
print ('and fps is vid_5 %d' %fps )

total frame in vid_5 is 858176
and fps is vid_5 30


In [5]:
#load the annotation data for the chosen person
vdata = np.load('/bigpool/UbiCompDataCleaned/P0'+ person +'AnnotationCleaned.npy')
print(vdata.shape)

(669140, 14)

In [6]:
#get the labels from the annotated data
vLabels = vdata[:,5:14].astype(int)
print(vLabels.shape)

(669140, 9)


In [7]:
#get the frame numbers from the annotated data
vFrameNos = vdata[:,0].astype(int)
print(vFrameNos.shape)

(669140,)


In [8]:
#get the timestamps from the annotated data
vTimestamp = vdata[:,1].astype(float)
vTimestamp = vTimestamp.astype(int)
print(vTimestamp.shape)

(669140,)


In [9]:
#combine the frame numbers, timestamps and labels of the chosen person into one numpy array
vdata_final = np.column_stack((vFrameNos, vTimestamp, vLabels))
print(vdata_final.shape)
print(vdata_final[:,0])

(669140, 11)


array([     0,      0,      1, ..., 858174, 858174, 858174])

In [11]:
#load the RecordingCleaned file of the chosen person to fetch the gaze data
gdata = np.load('/bigpool/UbiCompDataCleaned/P0'+ person +'RecordingCleaned.npy')
print(gdata)

array([[0.00000000e+00, 3.61449811e+04, 3.12154722e-01, ...,
        7.51687393e+01, 4.79616241e+01, 1.43582260e+02],
       [1.00000000e+00, 3.61450130e+04, 3.12439615e-01, ...,
        7.52819061e+01, 4.80286331e+01, 1.43923981e+02],
       [2.00000000e+00, 3.61450455e+04, 3.01962659e-01, ...,
        7.54529190e+01, 5.39445229e+01, 1.59977814e+02],
       ...,
       [7.25311000e+05, 6.73912231e+04, 2.85524412e-01, ...,
        9.06054077e+01, 6.35528831e+01, 1.37331528e+02],
       [7.25312000e+05, 6.73912591e+04, 2.85230136e-01, ...,
        9.05915222e+01, 6.37343254e+01, 1.36912201e+02],
       [7.25313000e+05, 6.73912918e+04, 2.84961953e-01, ...,
        9.08436508e+01, 6.39388428e+01, 1.36676147e+02]])

In [12]:
#obtain the timestamps from the RecordingCleaned file 
gTimestamp = gdata[:,1].astype(float)
gTimestamp = gTimestamp.astype(int)
print(gTimestamp.shape)
print(gTimestamp)

(669140,)


array([36144, 36145, 36145, ..., 67391, 67391, 67391])

In [13]:
#obtain the gazeX, gazeY, pupilX and pupilY for the chosen person
gGazeXY = gdata[:,2:6].astype(float)
print(gGazeXY.shape)

(669140, 4)


In [14]:
#combine timestamp and gaze data of the person into one numpy array
gdata_final = np.column_stack((gTimestamp, gGazeXY))
print(gdata_final.shape)
gdata_final[:,[0]]

(669140, 5)


array([[36144.],
       [36145.],
       [36145.],
       ...,
       [67391.],
       [67391.],
       [67391.]])

In [21]:
# Build the setup.py for optical flow image generation
!python3 setup.py build_ext -i

running build_ext


In [22]:
#function used to generate optical flow image
def opflow():
    !python3 demo.py -viz

In [23]:
#check if the HDF5 file already exists
if Path(HDF5_PATH).is_file():
    print("Open storage for appending.")
    hdf = h5py.File(HDF5_PATH, mode='a')
else:
    print("Creating new storage for writing.")
    hdf = h5py.File(HDF5_PATH, mode='w') 

Open storage for appending.


In [24]:
#function to write the sampled data from the video into HDF5 file
def add_to_hdf(hdf, rawImage, flowImage, gazeData, set_name, frame_no, index ,label_vals):
    groupName = set_name
    pGroup = None
    groupExists = "/" + groupName in hdf
    if not (groupExists):
        pGroup = hdf.create_group(groupName)
    else:
        pGroup = hdf["/" + groupName]
    
    if not (groupExists):
        pGroup.create_dataset("rawImages", data=rawImage, maxshape=(None, 224, 224, 3), chunks=True)
        pGroup.create_dataset("flowImages", data=flowImage, maxshape=(None, 224, 224, 3), chunks=True)  
        pGroup.create_dataset("gazeData", data=gazeData, maxshape=(None, 60), chunks=True) 
        pGroup.create_dataset("labels", data=label_vals, maxshape=(None, 9), chunks=True)
        print('All Groups created')
    else:
        pGroup["rawImages"].resize((pGroup["rawImages"].shape[0] + rawImage.shape[0]), axis = 0)
        pGroup["rawImages"][-rawImage.shape[0]:] = rawImage
        pGroup["flowImages"].resize((pGroup["flowImages"].shape[0] + flowImage.shape[0]), axis = 0)
        pGroup["flowImages"][-flowImage.shape[0]:] = flowImage
        pGroup["gazeData"].resize((pGroup["gazeData"].shape[0] + gazeData.shape[0]), axis = 0)
        pGroup["gazeData"][-gazeData.shape[0]:] = gazeData
        pGroup["labels"].resize((pGroup["labels"].shape[0] + label_vals.shape[0]), axis = 0)
        pGroup["labels"][-label_vals.shape[0]:] = label_vals
    
    print('Data from frame_no %s is written to hdf5 file whose index is %s' % (frame_no, index))

In [25]:
#function used to extract a particular frame using the frame number
def Extract_frame(frame_no):
    # To resize the image
    vid.set(1,frame_no)
    success,frame = vid.read()
    resized = cv2.resize(frame, (224,224))
    return resized

In [26]:
#function to generate optical flow image using the current and the previous image
def Opticalflow_gen(index, curr_frame):
    if(index < 299):
        prev_frame = curr_frame
    else:        
        #prev_frame = data.iloc[prev_index,0]
        prev_frame = framelist[-1]
        
    vid.set(1,prev_frame)
    success,frame = vid.read()
    resized_image = cv2.resize(frame, (224,224))
    cv2.imwrite("prev_frame.jpg", resized_image)
    vid.set(1,curr_frame)
    success,frame = vid.read()
    resized_image = cv2.resize(frame, (224,224))
    cv2.imwrite("curr_frame.jpg", resized_image)
    opflow()
    print('Optical flow image generated for frame_nos: %s and %s' % (prev_frame, curr_frame))

In [27]:
#function used to extract the generated optical flow image
def Extract_flow():
    img = cv2.imread('/home/gururajaramesh/outFlow_img.png')
    resizedImage = cv2.resize(img, (224,224))
    return resizedImage 

In [28]:
#obtain rawImage, flowImage and corresponding gazeData using timestamp and store the data in the HDF5 file
interval = 300
framelist = []
timestamplist = []
count = 0
for index in range(gdata_final.shape[0]):
    if(index%interval==0):
        gTimestamp = gdata_final[index,0]
        timestamplist.append(gTimestamp)
        gazeXY = gdata_final[index:(index+30),1:3]
        gazedata = gazeXY.reshape(-1,60)
        for i in range(vdata_final.shape[0]): 
            if(vdata_final[i,1]==gTimestamp):
                print('Timestep %s from gaze is found as %s in video data ' % (gTimestamp,vdata_final[i,1]))
                frame_no = vdata_final[i,0]
                label = vdata_final[i,2:11]
                label = label.reshape(-1,9)
                raw_img = Extract_frame(frame_no)
                reshapedRaw_img = np.reshape(raw_img,(-1,224,224,3))
                Opticalflow_gen(i, frame_no)
                flow_img = Extract_flow()
                reshapedFlow_img = np.reshape(flow_img,(-1,224,224,3))
                add_to_hdf(hdf, reshapedRaw_img, reshapedFlow_img, gazedata, "train", frame_no, i, label)
                framelist.append(frame_no)
                count = count + 1
                print('feature extracted for frame_no: ', frame_no)
                print('%s frames are finished' % count)
                break
framelist = np.asarray(framelist)
np.save('/home/gururajaramesh/Extracted Frame and Timestamp/ExtractedFrameList_P04.npy',framelist)
timestamplist = np.asarray(timestamplist)
np.save('/home/gururajaramesh/Extracted Frame and Timestamp/ExtractedTimestampList_P04.npy',timestamplist)

Timestep 36144.0 from gaze is found as 36144 in video data 
Constructing pyramid...done!
Pyramid level 7All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are inva

All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in 

All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in 

All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!

Pyramid level 2All the pixels

All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!
All the pixels are invalid in estimation Laplacian noise!!!
Something severely wrong happened!!!

Pyramid level 0All the pixels

...

Timestep 66231.0 from gaze is found as 66231 in video data 
Constructing pyramid...done!
Pyramid level 7
Pyramid level 6
Pyramid level 5
Pyramid level 4
Pyramid level 3
Pyramid level 2
Pyramid level 1
Pyramid level 0
Time Taken: 3.21 seconds for image of size (224, 224, 3)
Optical flow image generated for frame_nos: 825918 and 826309
Data from frame_no 826309 is written to hdf5 file whose index is 645892
feature extracted for frame_no:  826309
2154 frames are finished
Timestep 66243.0 from gaze is found as 66243 in video data 
Constructing pyramid...done!
Pyramid level 7
Pyramid level 6
Pyramid level 5
Pyramid level 4
Pyramid level 3
Pyramid level 2
Pyramid level 1
Pyramid level 0
Time Taken: 3.18 seconds for image of size (224, 224, 3)
Optical flow image generated for frame_nos: 826309 and 826647
Data from frame_no 826647 is written to hdf5 file whose index is 646189
feature extracted for frame_no:  826647
2155 frames are finished
Timestep 66255.0 from gaze is found as 66255 in video 

Pyramid level 3
Pyramid level 2
Pyramid level 1
Pyramid level 0
Time Taken: 3.13 seconds for image of size (224, 224, 3)
Optical flow image generated for frame_nos: 831829 and 832195
Data from frame_no 832195 is written to hdf5 file whose index is 650989
feature extracted for frame_no:  832195
2171 frames are finished
Timestep 66454.0 from gaze is found as 66454 in video data 
Constructing pyramid...done!
Pyramid level 7
Pyramid level 6
Pyramid level 5
Pyramid level 4
Pyramid level 3
Pyramid level 2
Pyramid level 1
Pyramid level 0
Time Taken: 2.70 seconds for image of size (224, 224, 3)
Optical flow image generated for frame_nos: 832195 and 832585
Data from frame_no 832585 is written to hdf5 file whose index is 651300
feature extracted for frame_no:  832585
2172 frames are finished
Timestep 66466.0 from gaze is found as 66466 in video data 
Constructing pyramid...done!
Pyramid level 7
Pyramid level 6
Pyramid level 5
Pyramid level 4
Pyramid level 3
Pyramid level 2
Pyramid level 1
Pyrami

Timestep 66672.0 from gaze is found as 66672 in video data 
Constructing pyramid...done!
Pyramid level 7
Pyramid level 6
Pyramid level 5
Pyramid level 4
Pyramid level 3
Pyramid level 2
Pyramid level 1
Pyramid level 0
Time Taken: 2.73 seconds for image of size (224, 224, 3)
Optical flow image generated for frame_nos: 838400 and 838711
Data from frame_no 838711 is written to hdf5 file whose index is 656376
feature extracted for frame_no:  838711
2189 frames are finished
Timestep 66683.0 from gaze is found as 66683 in video data 
Constructing pyramid...done!
Pyramid level 7
Pyramid level 6
Pyramid level 5
Pyramid level 4
Pyramid level 3
Pyramid level 2
Pyramid level 1
Pyramid level 0
Time Taken: 2.94 seconds for image of size (224, 224, 3)
Optical flow image generated for frame_nos: 838711 and 839031
Data from frame_no 839031 is written to hdf5 file whose index is 656682
feature extracted for frame_no:  839031
2190 frames are finished
Timestep 66695.0 from gaze is found as 66695 in video 

Pyramid level 3
Pyramid level 2
Pyramid level 1
Pyramid level 0
Time Taken: 2.84 seconds for image of size (224, 224, 3)
Optical flow image generated for frame_nos: 844480 and 844882
Data from frame_no 844882 is written to hdf5 file whose index is 661494
feature extracted for frame_no:  844882
2206 frames are finished
Timestep 66902.0 from gaze is found as 66902 in video data 
Constructing pyramid...done!
Pyramid level 7
Pyramid level 6
Pyramid level 5
Pyramid level 4
Pyramid level 3
Pyramid level 2
Pyramid level 1
Pyramid level 0
Time Taken: 2.61 seconds for image of size (224, 224, 3)
Optical flow image generated for frame_nos: 844882 and 845222
Data from frame_no 845222 is written to hdf5 file whose index is 661795
feature extracted for frame_no:  845222
2207 frames are finished
Timestep 66930.0 from gaze is found as 66930 in video data 
Constructing pyramid...done!
Pyramid level 7
Pyramid level 6
Pyramid level 5
Pyramid level 4
Pyramid level 3
Pyramid level 2
Pyramid level 1
Pyrami

Timestep 67228.0 from gaze is found as 67228 in video data 
Constructing pyramid...done!
Pyramid level 7
Pyramid level 6
Pyramid level 5
Pyramid level 4
Pyramid level 3
Pyramid level 2
Pyramid level 1
Pyramid level 0
Time Taken: 3.02 seconds for image of size (224, 224, 3)
Optical flow image generated for frame_nos: 853932 and 854317
Data from frame_no 854317 is written to hdf5 file whose index is 666900
feature extracted for frame_no:  854317
2224 frames are finished
Timestep 67245.0 from gaze is found as 67245 in video data 
Constructing pyramid...done!
Pyramid level 7
Pyramid level 6
Pyramid level 5
Pyramid level 4
Pyramid level 3
Pyramid level 2
Pyramid level 1
Pyramid level 0
Time Taken: 3.11 seconds for image of size (224, 224, 3)
Optical flow image generated for frame_nos: 854317 and 854704
Data from frame_no 854704 is written to hdf5 file whose index is 667196
feature extracted for frame_no:  854704
2225 frames are finished
Timestep 67261.0 from gaze is found as 67261 in video 

In [29]:
#close the HDF5 file
hdf.close()